In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import pickle
import time
import os
import json

from pandas import json_normalize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
from sklearn.preprocessing import normalize,StandardScaler
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.compose import ColumnTransformer


import librosa
from scipy.stats import skew

In [8]:
with open('pickles/df_librosa_mz.pkl', 'rb') as f:
    df_librosa_mz = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'pickles/df_librosa_mz.pkl'

In [6]:
df_moz = pd.DataFrame(pd.read_csv('datasets/cv-corpus-12.0-delta-2022-12-07/en/validated.tsv', sep= "\t"))

In [5]:
df_moz.isna().sum()

client_id        0
path             0
sentence         0
up_votes         0
down_votes       0
age            934
gender         934
accents        725
locale           0
segment       6717
dtype: int64

In [6]:
df_librosa_mz = pd.DataFrame(columns=['filename',
                                   'mean_mfccs',
                                   'max_mfccs',
                                   'min_mfccs',
                                   'skew_mfccs',
                                   'variance_mfccs',
                                   'mean_spectral_centroids',
                                   'min_spectral_centroids',
                                   'max_spectral_centroids',
                                   'variance_spectral_centroids',
                                   'skew_spectral_centroids',
                                   'mean_spectral_bandwidth',
                                   'mean_spectral_contrast',
                                   'zero_crossing_rate',
                                   'chroma_cens'
                                   ])

In [7]:
folder_path = 'datasets/cv-corpus-11.0-delta-2022-09-21/en/clips/'

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path,filename)
    if os.path.isfile(file_path):
        y,sr = librosa.load(file_path,sr=44100)
        y = librosa.effects.preemphasis(y, coef=0.97)
        # mfccs
        mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
        mean_mfccs = mfccs.mean()
        max_mfccs = mfccs.max()
        min_mfccs = mfccs.min()
        skew_mfccs = skew(mfccs)
        variance_mfccs = mfccs.var()
        #spectral centroids
        spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        mean_spectral_centroids = spectral_centroids.mean()
        max_spectral_centroids = spectral_centroids.max()
        min_spectral_centroids = spectral_centroids.min()
        skew_spectral_centroids = skew(spectral_centroids)
        variance_spectral_centroids = spectral_centroids.var()
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
        mean_spectral_bandwidth = spectral_bandwidth.mean()
        spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)[0]
        mean_spectral_contrast = spectral_contrast.mean()
        zero_crossing_rate = librosa.feature.zero_crossing_rate(y)[0].mean()
        chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr)[0]
        chroma_cens_mean = chroma_cens.mean()
        
        librosa_dict = {
            'filename': filename,
            'mean_mfccs': mean_mfccs,
            'max_mfccs': max_mfccs,
            'min_mfccs': min_mfccs,
            'skew_mfccs': skew_mfccs,
            'variance_mfccs': variance_mfccs,
            'mean_spectral_centroids': mean_spectral_centroids,
            'max_spectral_centroids': max_spectral_centroids,
            'min_spectral_centroids': min_spectral_centroids,
            'skew_spectral_centroids': skew_spectral_centroids,
            'variance_spectral_centroids': variance_spectral_centroids,
            'mean_spectral_bandwidth': mean_spectral_bandwidth,
            'mean_spectral_contrast': mean_spectral_contrast,
            'zero_crossing_rate': zero_crossing_rate,
            'chroma_cens': chroma_cens_mean
        }
#         df_librosa_mz = df_librosa_mz.append(librosa_dict, ignore_index=True)

/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[-5.7964370e-15  1.0057512e-14 -3.0596829e-15 ...  8.9743259e-07
  3.9539245e-06  5.9196964e-07], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[-2.6238667e-15 -2.9959115e-15  3.5193467e-14 ... -2.0066705e-06
 -4.4057674e-07  5.1514945e-07], sr=44100 as keyword args. From version 0.10 passing these as

/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -8.1176986e-06
 -7.5488897e-06  5.4288339e-05], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[-3.3941464e-13  4.1976237e-13 -2.7174443e-14 ... -3.7609357e-07
  8.1880188e-07  5.6548413e-07], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  7.20283388e-09
  2.75229883e-09

/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 2.2596118e-12 -2.5853935e-12 -1.9727078e-12 ...  8.7091385e-06
  8.0250629e-06 -1.6255988e-05], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 3.1708940e-13 -6.0323795e-13  1.3328225e-12 ... -3.3026328e-05
  4.4262677e-05  5.6431402e-04], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 6.6809327e-12 -7.2527661e-12 -7.2039258e-12 ... -1.7172979e-05
 -1.6619926e-05  8.6

/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 3.4412157e-14 -3.7990044e-14 -1.3939045e-14 ... -9.7607835e-07
 -1.2514265e-06 -1.3209996e-06], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 1.5378628e-12 -2.4214736e-12  2.0043429e-12 ...  7.9789871e-08
  2.0957756e-07 -8.5511935e-08], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  4.4237247e-08
  3.5945764e-08 -3.6

/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[-1.0462114e-12  1.1361266e-12  9.1436450e-13 ...  3.6354237e-08
  1.3519455e-06  1.5102580e-07], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 7.5290062e-13 -8.4451055e-13 -5.4838019e-13 ...  7.6788933e-07
  3.2861365e-06 -3.2692503e-06], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 4.3516566e-12 -4.9248028e-12 -2.7027898e-12 ... -2.5783975e-06
  7.6208398e-06  2.0

...

/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  1.1483611e-05
 -7.4048876e-06 -4.8531747e-05], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[-1.8611050e-16  2.2573460e-16  1.3506197e-16 ... -6.1390841e-07
 -7.4891636e-07  5.7589114e-07], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 1.5650631e-12 -2.0198420e-12 -1.0213059e-12 ... -9.9066028e-08
 -1.7026196e-07  2.9

/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 5.6176964e-12 -7.1076231e-12 -1.7941187e-12 ...  1.9903368e-09
 -7.9938545e-09  7.9382360e-08], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 8.6560140e-14 -1.1714169e-13  1.0456641e-13 ... -2.3047585e-05
 -2.2460636e-05  3.2504613e-05], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 1.0469056e-15  2.5497381e-14  3.0261170e-14 ...  1.1927825e-06
  8.0885496e-07 -6.9

/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[-3.4416785e-14  3.2711731e-14  6.1061751e-14 ... -5.8992001e-08
 -7.2612934e-08  1.1010997e-08], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -9.7321774e-08
 -9.0538826e-08  1.7097751e-07], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 7.5826753e-13 -1.1027029e-12 -5.1588302e-13 ...  4.0092600e-06
  1.1293486e-05  5.1

/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  1.0392251e-05
 -8.8336510e-06  1.5682512e-05], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[-6.3581041e-13  7.2501293e-13  3.1840172e-13 ...  7.9129973e-07
 -7.5134255e-07 -1.2335355e-06], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[-7.1655613e-14  9.3333747e-14 -9.2981761e-14 ... -6.7818029e-08
  1.3624484e-07  7.8

/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 1.2490344e-12 -1.3752860e-12 -1.1830939e-12 ... -9.3156941e-06
  1.4672982e-05 -3.1537056e-05], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 1.6024744e-12 -1.9190029e-12 -4.3089772e-13 ...  6.5839868e-07
  5.7671843e-07  3.0452404e-07], sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = (librosa.feature.mfcc(y, sr, n_mels=40, n_fft=2048, hop_length=512, win_length=1024)[0])
/var/folders/fm/_6kg8xkn3jz4slz27zwpqnhc0000gn/T/ipykernel_2549/1092995422.py:9: FutureWarning: Pass y=[ 1.6024744e-12 -1.9190029e-12 -4.3089772e-13 ...  6.5839868e-07
  5.7671843e-07  3.0

In [1]:
df_librosa_mz

NameError: name 'df_librosa_mz' is not defined

In [9]:
with open('pickles/df_librosa.pkl', 'rb') as f:
    df_librosa = pickle.load(f)
    
with open('pickles/df.pkl', 'rb') as f:
    df = pickle.load(f)

In [10]:
df_unique = df.drop(columns = ['listenerId','utteranceId','systemId','sentenceId','filename','testpageId','MOS','locale','choice','isNative','wrongValidation','lowNatural','sameScores','highSynthetic','clean','listenerReliability'])

In [ ]:
df_unique = df_unique.drop_duplicates()

In [ ]:
df_unique

In [ ]:
df_librosa_mz['modelId'] = 'm0'

In [ ]:
df_all = pd.concat([df_unique,df_librosa_mz],axis=0)

In [ ]:
df_all

In [ ]:
# 0 is generated, 1 is human
df_all['modelId'] = np.where(df_all['modelId'] != 'm0',0,1)

In [ ]:
df_all['modelId'].value_counts()

## Plotting Values for df_all (similar to worksheet 4)

In [11]:
generated = df_all[df_all['modelId'] != 1]
human = df_all[df_all['modelId'] == 1]

In [12]:
plt.figure(figsize=(10, 4))
plt.plot(generated['mean_mfccs'], label='generated MFCCs Mean')
plt.plot(human['mean_mfccs'], label='Human MFCCs Mean')
plt.ylabel('MFCCs')
plt.xlabel('iteration')

plt.legend()
plt.tight_layout()
plt.show()

In [13]:
plt.figure(figsize=(10, 4))
plt.plot(generated['min_mfccs'], label='generated MFCCs Min')
plt.plot(human['min_mfccs'], label='Human MFCCs Min')
plt.ylabel('min MFCCs')
plt.xlabel('iteration')

plt.legend()
plt.tight_layout()
plt.show()

In [14]:
plt.figure(figsize=(10, 4))
plt.plot(generated['mean_spectral_centroids'], label='generated SC ')
plt.plot(human['mean_spectral_centroids'], label='Human SC ')
plt.ylabel('Mean Spectral Centroids')
plt.xlabel('iteration')

plt.legend()
plt.tight_layout()
plt.show()

In [15]:
df_moz

In [16]:
char_counts = df_moz['sentence'].str.len()
char_counts.sum()